In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
from sklearn.metrics import roc_auc_score,accuracy_score

In [3]:
from scipy.special import cbrt
import re

In [4]:
os.chdir('../pickles')

In [5]:
numerical = pickle.load(open('numerical.pickle','rb'))
categorical = pickle.load(open('categorical.pickle','rb'))
conversion_dict = pickle.load(open('conversion_dict.pickle','rb'))
imputation_cols = pickle.load(open('imputation_cols.pickle','rb'))
model_columns = pickle.load(open('model_columns','rb'))
lr_model_columns = pickle.load(open('lr_model_columns.pickle','rb'))
order = pickle.load(open('order.pickle','rb'))
scale = pickle.load(open('scale.pickle','rb'))
pca = pickle.load(open('pca.pickle','rb'))

In [6]:
os.chdir('../Input_Data')

In [7]:
df_val = pd.read_csv('validation.csv',header=0)

In [8]:
rename_dict = {col : col.replace('%','pct_') for col in df_val.columns if '%' in col}

In [9]:
df_val.rename(rename_dict,axis=1,inplace=True)

In [10]:
rename_dict={}
for col in order.keys():
    df_val[col]=df_val[col]/10**order[col]
    rename_dict[col] = col+"_order{}".format(order[col])

In [11]:
df_val.rename(rename_dict,axis=1,inplace=True)

In [12]:
for col in categorical:
    if len(conversion_dict[col])==1:
        category = conversion_dict[col][0]
        df_val[col+'_dum_'+str(category)] = 0
        df_val.loc[df_val[col]==category,col+'_dum_'+str(category)]=1
    else:
        total_categories = len(conversion_dict[col])
        dummies = len(str(int(bin(total_categories)[2:],10)))
        bin_conv=[]
        for i in range(total_categories):
            bin_conv.append(conversion_dict[col][i][1])
        for j in range(dummies):
            df_val[col+'_dum_'+str(j)]=0
            for i,cat in enumerate([conv[0] for conv in conversion_dict[col]]):
                df_val.loc[df_val[col]==cat,col+'_dum_'+str(j)]=bin_conv[i]%10
                bin_conv[i]=bin_conv[i]//10
    df_val.drop(col,axis=1,inplace=True)
    print(col+' done')

Sector done


In [13]:
os.chdir('../Statistics/')

In [14]:
edd_df = pd.read_excel('report.xlsx',sheet_name='edd_v01',header=0)

In [15]:
transform_dict = {'log':lambda x: np.log(x),'sqr':lambda x: x**2,'sqrt':lambda x: np.sqrt(x),'exp':lambda x:np.exp(x),
                 'cube':lambda x: x**3,'cuberoot': lambda x: cbrt(x)}

In [16]:
for col in list(edd_df['Var']):
    applied = edd_df.loc[edd_df['Var']==col,'conversions'].values[0]
    if applied != '' and applied != 'categorical':
        try:
            for t in applied.split(','):
                df_val[col+'_'+t] = df_val[col].apply(transform_dict[t])
        except:
            pass

In [17]:
os.chdir('../Imputation_models')

In [18]:
for col in [x for x in numerical if x!='lift_ind']:
    model = pickle.load(open(col+'_impute.pickle','rb'))
    if df_val[col].isnull().any():
        indices = df_val.loc[df_val[col].isnull()].index.tolist()
        df_val.loc[indices,col]=np.array(model.predict(np.array(df_val.loc[indices,imputation_cols])))
        del indices
    del model
    print(col+' imputed')

Prev_Close_order5 imputed
High_1d_order5 imputed
Low_1d_order5 imputed
Last_1d_order5 imputed
VWAP_1d_order5 imputed
Volume_1d_order9 imputed
Turnover_1d_order11 imputed
Trades_1d_order7 imputed
Deliverable_Volume_1d_order9 imputed
pct_Deliverble_1d imputed
Open_nifty_1d_order5 imputed
High_nifty_1d_order5 imputed
Low_nifty_1d_order5 imputed
Close_nifty_1d_order5 imputed
Volume_nifty_1d_order9 imputed
Turnover_nifty_1d_order11 imputed
Open_auto_1d_order5 imputed
High_auto_1d_order5 imputed
Low_auto_1d_order5 imputed
Close_auto_1d_order5 imputed
Open_bank_1d_order5 imputed
High_bank_1d_order5 imputed
Low_bank_1d_order5 imputed
Close_bank_1d_order5 imputed
Open_fmcg_1d_order5 imputed
High_fmcg_1d_order5 imputed
Low_fmcg_1d_order5 imputed
Close_fmcg_1d_order5 imputed
Open_it_1d_order5 imputed
High_it_1d_order5 imputed
Low_it_1d_order5 imputed
Close_it_1d_order5 imputed
Open_media_1d_order4 imputed
High_media_1d_order4 imputed
Low_media_1d_order4 imputed
Close_media_1d_order4 imputed
Open_

In [19]:
x_val,y_val = np.array(df_val[lr_model_columns]),np.array(df_val['lift_ind'])
x,y = np.array(df_val[model_columns]),np.array(df_val['lift_ind'])
x_pca,y_pca = pca.transform(scale.transform(np.array(df_val[model_columns]))),np.array(df_val['lift_ind'])

In [20]:
os.chdir('../Models')

In [21]:
models = {'LR':pickle.load(open('LR.pickle','rb')),
                      'RF':pickle.load(open('RF.pickle','rb')),
                    'GBM':pickle.load(open('GBM.pickle','rb')),
         'RF_full':pickle.load(open('RF_full.pickle','rb')),
         'GBM_full':pickle.load(open('GBM_full.pickle','rb')),
          'RF_pca' : pickle.load(open('RF_pca.pickle','rb')),
          'GBM_pca' : pickle.load(open('GBM_pca.pickle','rb'))
         }


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [22]:
df_val['RF_Prediction'] = models['RF'].predict_proba(x_val)[:,1]

In [23]:
df_val['GBM_Prediction'] = models['GBM'].predict_proba(x_val)[:,1]

In [24]:
df_val['LR_Prediction'] = models['LR'].predict_proba(df_val[lr_model_columns])[:,1]

In [25]:
df_val['RF_full_prediction'] = models['RF_full'].predict_proba(df_val[model_columns])[:,1]

In [26]:
df_val['GBM_full_prediction'] = models['GBM_full'].predict_proba(df_val[model_columns])[:,1]

In [32]:
df_val['RF_pca_prediction'] = models['RF_pca'].predict_proba(x_pca)[:,1]

In [30]:
df_val['GBM_pca_prediction'] = models['GBM_pca'].predict_proba(x_pca)[:,1]

In [34]:
scores = [roc_auc_score(df_val['lift_ind'],df_val['LR_Prediction']),roc_auc_score(df_val['lift_ind'],df_val['RF_Prediction']),roc_auc_score(df_val['lift_ind'],df_val['GBM_Prediction']),
         roc_auc_score(df_val['lift_ind'],df_val['RF_full_prediction']),roc_auc_score(df_val['lift_ind'],df_val['GBM_full_prediction']),
         roc_auc_score(df_val['lift_ind'],df_val['RF_pca_prediction']),roc_auc_score(df_val['lift_ind'],df_val['GBM_pca_prediction'])]

In [63]:
df_scores = pd.DataFrame(list(zip(['LR','RF','GBM','RF_full','GBM_full','RF_pca','GBM_pca'],scores)),columns=['models','roc_auc_score'])

In [59]:
df_scores

,models,roc_auc_score
0,LR,0.481843
1,RF,0.520416
2,GBM,0.524365
3,RF_full,0.544155
4,GBM_full,0.539097
5,RF_pca,0.507370
6,GBM_pca,0.484264


In [37]:
df_val['Ensembled_Prob'] = (.48*df_val['LR_Prediction']+.52*df_val['RF_Prediction']+.52*df_val['GBM_Prediction']+
                           .54*df_val['RF_full_prediction']+.54*df_val['GBM_full_prediction']
                           +.51*df_val['RF_pca_prediction']+.48*df_val['GBM_pca_prediction'])/(1.52+1.08+.99)

In [38]:
roc_auc_score(df_val['lift_ind'],df_val['Ensembled_Prob'])

0.526332128263197

In [39]:
from openpyxl import load_workbook,Workbook

In [40]:
os.chdir('../Statistics/')

In [12]:
rf_importance = pd.DataFrame(list(zip(lr_model_columns,list(models['RF'].feature_importances_))),columns=['Var','Importance'])
book = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

rf_importance.to_excel(writer, "rf_feature_importance", index=False)

writer.save()

In [13]:
gbm_importance = pd.DataFrame(list(zip(lr_model_columns,list(models['GBM'].feature_importances_))),columns=['Var','Importance'])
book = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

gbm_importance.to_excel(writer, "gbm_feature_importance", index=False)

writer.save()

In [14]:
linear_importance = pd.DataFrame(list(zip([1]+list(lr_model_columns),[models['LR'].intercept_[0]]+list(models['LR'].coef_[0]))),columns=['Var','coeff'])
book = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

linear_importance.to_excel(writer, "linear_model_coef", index=False)

writer.save()

In [41]:
rf_importance = pd.DataFrame(list(zip(model_columns,list(models['RF_full'].feature_importances_))),columns=['Var','Importance'])
book = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

rf_importance.to_excel(writer, "rf_full_feature_importance", index=False)

writer.save()

In [42]:
gbm_importance = pd.DataFrame(list(zip(model_columns,list(models['GBM_full'].feature_importances_))),columns=['Var','Importance'])
book = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

gbm_importance.to_excel(writer, "gbm_full_feature_importance", index=False)

writer.save()

In [43]:
df_val['ensembled_prob2'] = (df_val['RF_full_prediction']+df_val['GBM_full_prediction'])/2

In [44]:
roc_auc_score(df_val['lift_ind'],df_val['ensembled_prob2'])

0.5413460665498274

In [64]:
df_scores = df_scores.append(pd.DataFrame([['ensembled_all',0.526332128263197],
                                          ['ensembled_full',0.5413460665498274]],columns=df_scores.columns),ignore_index=True)

In [65]:
df_scores

,models,roc_auc_score
0,LR,0.481843
1,RF,0.520416
2,GBM,0.524365
3,RF_full,0.544155
4,GBM_full,0.539097
5,RF_pca,0.507370
6,GBM_pca,0.484264
7,ensembled_all,0.526332
8,ensembled_full,0.541346


In [66]:
book = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

df_scores.to_excel(writer, "model_scores", index=False)

writer.save()

In [67]:
def model_advanced_statistics(model_name,x_val,y_val):
    y_prob = models[model_name].predict_proba(x_val)[:,1]
    sorted_probs = sorted(list(zip(y_val,y_prob)),key = lambda x: x[1] , reverse = True)
    obs = len(list(y_val))//100
    base_df = pd.DataFrame(sorted_probs,columns=['label','prob'])
    pos,neg = sum(list(y_val)),len(list(y_val))-sum(list(y_val))
    base_df['Bucket'] = base_df.index//obs
    bucket_list = base_df.groupby(by='Bucket',axis=0).agg({'label' : ['count','sum'],'prob' : ['max','min','mean']})
    bucket_list.columns = bucket_list.columns.droplevel()
    bucket_list.columns = ['observations','true_positives','prob_max','prob_min','prob_mean']
    bucket_list['cum_observations'] = bucket_list['observations'].cumsum()
    bucket_list['cum_true_pos'] = bucket_list['true_positives'].cumsum()
    bucket_list['coverage'] = bucket_list['cum_true_pos']/list(bucket_list['cum_true_pos'])[-1]
    bucket_list['cum_true_pct'] = bucket_list['cum_true_pos']/bucket_list['cum_observations']
    return bucket_list

In [69]:
RF_Stats = model_advanced_statistics('RF',x_val,y_val)
GBM_Stats = model_advanced_statistics('GBM',x_val,y_val)
LR_Stats = model_advanced_statistics('LR',x_val,y_val)
RF_full_stats = model_advanced_statistics('RF_full',x,y)
GBM_full_stats = model_advanced_statistics('GBM_full',x,y)
RF_pca_stats = model_advanced_statistics('RF_pca',x_pca,y_pca)
GBM_pca_stats = model_advanced_statistics('GBM_pca',x_pca,y_pca)

In [70]:
for stats,name in [(RF_Stats,'RF'),(GBM_Stats,'GBM'),(LR_Stats,'LR'),(RF_full_stats,'RF_full'),
                  (GBM_full_stats,'GBM_full'),(RF_pca_stats,'RF_pca'),(GBM_pca_stats,'GBM_pca')]:
    book = load_workbook('report.xlsx')
    writer = pd.ExcelWriter('report.xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    stats.to_excel(writer, name+'_stats')

    writer.save()

In [75]:
def model_stats_ensemble(y_prob,y_val):
    y_prob = y_prob
    sorted_probs = sorted(list(zip(y_val,y_prob)),key = lambda x: x[1] , reverse = True)
    obs = len(list(y_val))//100
    base_df = pd.DataFrame(sorted_probs,columns=['label','prob'])
    pos,neg = sum(list(y_val)),len(list(y_val))-sum(list(y_val))
    base_df['Bucket'] = base_df.index//obs
    bucket_list = base_df.groupby(by='Bucket',axis=0).agg({'label' : ['count','sum'],'prob' : ['max','min','mean']})
    bucket_list.columns = bucket_list.columns.droplevel()
    bucket_list.columns = ['observations','true_positives','prob_max','prob_min','prob_mean']
    bucket_list['cum_observations'] = bucket_list['observations'].cumsum()
    bucket_list['cum_true_pos'] = bucket_list['true_positives'].cumsum()
    bucket_list['coverage'] = bucket_list['cum_true_pos']/list(bucket_list['cum_true_pos'])[-1]
    bucket_list['cum_true_pct'] = bucket_list['cum_true_pos']/bucket_list['cum_observations']
    return bucket_list

In [76]:
ensembled_all_stats = model_stats_ensemble(df_val['Ensembled_Prob'],df_val['lift_ind'])
ensembled_full_stats = model_stats_ensemble(df_val['ensembled_prob2'],df_val['lift_ind'])

In [78]:
ensembled_full_stats

,observations,true_positives,prob_max,prob_min,prob_mean,cum_observations,cum_true_pos,coverage,cum_true_pct
Bucket,,,,,,,,,
0,31,19,0.542088,0.534185,0.537300,31,19,0.013204,0.612903
1,31,19,0.533875,0.518592,0.526375,62,38,0.026407,0.612903
2,31,19,0.517997,0.511987,0.514718,93,57,0.039611,0.612903
3,31,22,0.511928,0.510453,0.511266,124,79,0.054899,0.637097
4,31,23,0.510436,0.508799,0.509620,155,102,0.070883,0.658065
5,31,16,0.508729,0.506797,0.507731,186,118,0.082001,0.634409
6,31,15,0.506788,0.505603,0.506139,217,133,0.092425,0.612903
7,31,20,0.505590,0.504367,0.504857,248,153,0.106324,0.616935
8,31,12,0.504347,0.503150,0.503693,279,165,0.114663,0.591398


In [79]:
os.chdir('../output_data/')

In [80]:
df_val.columns

Index(['Symbol', 'Date', 'Series', 'Prev_Close_order5', 'High_1d_order5',
       'Low_1d_order5', 'Last_1d_order5', 'VWAP_1d_order5', 'Volume_1d_order9',
       'Turnover_1d_order11',
       ...
       'Voltality_500_log', 'RF_Prediction', 'GBM_Prediction', 'LR_Prediction',
       'RF_full_prediction', 'GBM_full_prediction', 'GBM_pca_prediction',
       'RF_pca_prediction', 'Ensembled_Prob', 'ensembled_prob2'],
      dtype='object', length=228)

In [81]:
df_val.to_csv('validation_predicted.csv',index=False)

In [82]:
os.chdir('../Statistics/')

In [83]:
df_val.to_excel('future_predictions.xlsx',sheet_name='data',index=False)